In [50]:
from pinecone import Pinecone, ServerlessSpec
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_pinecone import PineconeVectorStore
import os

os.environ["PINECONE_API_KEY"]="7c6028ed-8d2a-4003-8e7f-fca445aad9a1"
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

In [3]:
# Load the embedding model 
model_name="BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Chringo\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chringo\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [5]:
index_name = 'langchain-retrieval-augmentation-fast'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    )

In [4]:
embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-m3', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='')

In [11]:
# Load PDF gpt technical report and split
loader = PyPDFLoader("gpt-4.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=55)
texts = text_splitter.split_documents(documents)

In [59]:
len(texts)

671

In [9]:
print(pc.Index(index_name).describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [51]:
# Connect to Pinecone index and insert the chunked docs as contents
docsearch = PineconeVectorStore.from_documents(texts, embeddings, index_name=index_name)

In [52]:
print(pc.Index(index_name).describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 671}},
 'total_vector_count': 671}


In [61]:
query = "Was macht GPT4 besser als GPT3 und GPT3.5="


In [63]:
docsearch.similarity_search_with_score(query, k=4)

[(Document(page_content='GPT-4 signiﬁcantly outperforms both GPT-3.5 and Askell et al [100].ﬁxes to plot legend and title\n65', metadata={'page': 64.0, 'source': 'gpt-4.pdf'}),
  0.673246682),
 (Document(page_content='gpt-4 (no vision)\ngpt3.5Figure 4. GPT performance on academic and professional exams. In each case, we simulate the\nconditions and scoring of the real exam. Exams are ordered from low to high based on GPT-3.5\nperformance. GPT-4 outperforms GPT-3.5 on most exams tested. To be conservative we report the\nlower end of the range of percentiles, but this creates some artifacts on the AP exams which have very', metadata={'page': 5.0, 'source': 'gpt-4.pdf'}),
  0.656640768),
 (Document(page_content='GPT-4 signiﬁcantly reduces hallucinations relative to previous GPT-3.5 models (which have them-\nselves been improving with continued iteration). GPT-4 scores 19 percentage points higher than our\nlatest GPT-3.5 on our internal, adversarially-designed factuality evaluations (Figur

In [73]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [74]:
retriever.invoke(query)

[Document(page_content='GPT-4 signiﬁcantly outperforms both GPT-3.5 and Askell et al [100].ﬁxes to plot legend and title\n65', metadata={'page': 64.0, 'source': 'gpt-4.pdf'}),
 Document(page_content='gpt-4 (no vision)\ngpt3.5Figure 4. GPT performance on academic and professional exams. In each case, we simulate the\nconditions and scoring of the real exam. Exams are ordered from low to high based on GPT-3.5\nperformance. GPT-4 outperforms GPT-3.5 on most exams tested. To be conservative we report the\nlower end of the range of percentiles, but this creates some artifacts on the AP exams which have very', metadata={'page': 5.0, 'source': 'gpt-4.pdf'}),
 Document(page_content='GPT-4 signiﬁcantly reduces hallucinations relative to previous GPT-3.5 models (which have them-\nselves been improving with continued iteration). GPT-4 scores 19 percentage points higher than our\nlatest GPT-3.5 on our internal, adversarially-designed factuality evaluations (Figure 6).\nlearning technology writing 

In [91]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="llama3")
system_prompt = ("You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# /docs/concepts/#langchain-expression-language-lcel
chain = prompt | llm | StrOutputParser()

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
#print(chain.invoke({"topic": "Space travel"}))

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Was macht GPT4 besser als GPT3 und GPT3.5?"})
#response = rag_chain.invoke({"input": "Wie ist das Wetter heute?"})

print(response["answer"])

According to the text, GPT-4 outperforms both GPT-3.5 and Askell et al on most exams tested, significantly reduces hallucinations compared to previous GPT-3.5 models, and scores 19 percentage points higher than the latest GPT-3.5 on internal factuality evaluations.
